In [2]:
!jt -t grade3

# Pràctica bloc II (Inteligència Artificial)
#### Lluís Camino Pérez

La pràctica consisteix en general un model d'aprenentatge automàtic que sigui capaç de predir el valor de mercat d'un jugador de futbol donades les seves característiques.

Per a poder generar aquest model, es compta en la base de dades completa del joc de futbol FIFA, que conté la informació, així com el seu valor econòmic d'un total de $18207$ jugadors.

## Importació de llibreries

En primer lloc, hem d'importar les llibreries necessàries, que en aquest cas són les de `sklearn`, `pandas` i `numpy`.

In [3]:
import os

from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

import pandas as pd
import numpy as np

## Descripció de les dades

Per poder fer una anàlisi de les dades que tenim, en primer lloc les importarem i veurem una descripció d'aquestes.

In [4]:
df = pd.read_csv(os.path.join("..", "in", "fifa.csv"))

In [5]:
pd.set_option("display.max_columns", None)
df.head()

,Unnamed: 0,ID,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,Club Logo,Value,Wage,Special,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Work Rate,Body Type,Real Face,Position,Jersey Number,Joined,Loaned From,Contract Valid Until,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
0,0,158023,L. Messi,31,https://cdn.sofifa.org/players/4/19/158023.png,Argentina,https://cdn.sofifa.org/flags/52.png,94,94,FC Barcelona,https://cdn.sofifa.org/teams/2/light/241.png,€110.5M,€565K,2202,Left,5.0,4.0,4.0,Medium/ Medium,Messi,Yes,RF,10.0,"Jul 1, 2004",NaN,2021,5'7,159lbs,88+2,88+2,88+2,92+2,93+2,93+2,93+2,92+2,93+2,93+2,93+2,91+2,84+2,84+2,84+2,91+2,64+2,61+2,61+2,61+2,64+2,59+2,47+2,47+2,47+2,59+2,84.0,95.0,70.0,90.0,86.0,97.0,93.0,94.0,87.0,96.0,91.0,86.0,91.0,95.0,95.0,85.0,68.0,72.0,59.0,94.0,48.0,22.0,94.0,94.0,75.0,96.0,33.0,28.0,26.0,6.0,11.0,15.0,14.0,8.0,€226.5M
1,1,20801,Cristiano Ronaldo,33,https://cdn.sofifa.org/players/4/19/20801.png,Portugal,https://cdn.sofifa.org/flags/38.png,94,94,Juventus,https://cdn.sofifa.org/teams/2/light/45.png,€77M,€405K,2228,Right,5.0,4.0,5.0,High/ Low,C. Ronaldo,Yes,ST,7.0,"Jul 10, 2018",NaN,2022,6'2,183lbs,91+3,91+3,91+3,89+3,90+3,90+3,90+3,89+3,88+3,88+3,88+3,88+3,81+3,81+3,81+3,88+3,65+3,61+3,61+3,61+3,65+3,61+3,53+3,53+3,53+3,61+3,84.0,94.0,89.0,81.0,87.0,88.0,81.0,76.0,77.0,94.0,89.0,91.0,87.0,96.0,70.0,95.0,95.0,88.0,79.0,93.0,63.0,29.0,95.0,82.0,85.0,95.0,28.0,31.0,23.0,7.0,11.0,15.0,14.0,11.0,€127.1M
2,2,190871,Neymar Jr,26,https://cdn.sofifa.org/players/4/19/190871.png,Brazil,https://cdn.sofifa.org/flags/54.png,92,93,Paris Saint-Germain,https://cdn.sofifa.org/teams/2/light/73.png,€118.5M,€290K,2143,Right,5.0,5.0,5.0,High/ Medium,Neymar,Yes,LW,10.0,"Aug 3, 2017",NaN,2022,5'9,150lbs,84+3,84+3,84+3,89+3,89+3,89+3,89+3,89+3,89+3,89+3,89+3,88+3,81+3,81+3,81+3,88+3,65+3,60+3,60+3,60+3,65+3,60+3,47+3,47+3,47+3,60+3,79.0,87.0,62.0,84.0,84.0,96.0,88.0,87.0,78.0,95.0,94.0,90.0,96.0,94.0,84.0,80.0,61.0,81.0,49.0,82.0,56.0,36.0,89.0,87.0,81.0,94.0,27.0,24.0,33.0,9.0,9.0,15.0,15.0,11.0,€228.1M
3,3,193080,De Gea,27,https://cdn.sofifa.org/players/4/19/193080.png,Spain,https://cdn.sofifa.org/flags/45.png,91,93,Manchester United,https://cdn.sofifa.org/teams/2/light/11.png,€72M,€260K,1471,Right,4.0,3.0,1.0,Medium/ Medium,Lean,Yes,GK,1.0,"Jul 1, 2011",NaN,2020,6'4,168lbs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.0,13.0,21.0,50.0,13.0,18.0,21.0,19.0,51.0,42.0,57.0,58.0,60.0,90.0,43.0,31.0,67.0,43.0,64.0,12.0,38.0,30.0,12.0,68.0,40.0,68.0,15.0,21.0,13.0,90.0,85.0,87.0,88.0,94.0,€138.6M
4,4,192985,K. De Bruyne,27,https://cdn.sofifa.org/players/4/19/192985.png,Belgium,https://cdn.sofifa.org/flags/7.png,91,92,Manchester City,https://cdn.sofifa.org/teams/2/light/10.png,€102M,€355K,2281,Right,4.0,5.0,4.0,High/ High,Normal,Yes,RCM,7.0,"Aug 30, 2015",NaN,2023,5'11,154lbs,82+3,82+3,82+3,87+3,87+3,87+3,87+3,87+3,88+3,88+3,88+3,88+3,87+3,87+3,87+3,88+3,77+3,77+3,77+3,77+3,77+3,73+3,66+3,66+3,66+3,73+3,93.0,82.0,55.0,92.0,82.0,86.0,85.0,83.0,91.0,91.0,78.0,76.0,79.0,91.0,77.0,91.0,63.0,90.0,75.0,91.0,76.0,61.0,87.0,94.0,79.0,88.0,68.0,58.0,51.0,15.0,13.0,5.0,10.0,13.0,€196.4M


També podem tenir una visió general de les dades amb la instrucció ``describe``, que ens mostra un resum de l'estructura de les dades.

In [6]:
df.describe()

,Unnamed: 0,ID,Age,Overall,Potential,Special,International Reputation,Weak Foot,Skill Moves,Jersey Number,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes
count,18207.000000,18207.000000,18207.000000,18207.000000,18207.000000,18207.000000,18159.000000,18159.000000,18159.000000,18147.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000
mean,9103.000000,214298.338606,25.122206,66.238699,71.307299,1597.809908,1.113222,2.947299,2.361308,19.546096,49.734181,45.550911,52.298144,58.686712,42.909026,55.371001,47.170824,42.863153,52.711933,58.369459,64.614076,64.726967,63.503607,61.836610,63.966573,55.460047,65.089432,63.219946,65.311967,47.109973,55.868991,46.698276,49.958478,53.400903,48.548598,58.648274,47.281623,47.697836,45.661435,16.616223,16.391596,16.232061,16.388898,16.710887
std,5256.052511,29965.244204,4.669943,6.908930,6.136496,272.586016,0.394031,0.660456,0.756164,15.947765,18.364524,19.525820,17.379909,14.699495,17.694408,18.910371,18.395264,17.478763,15.327870,16.686595,14.927780,14.649953,14.766049,9.010464,14.136166,17.237958,11.820044,15.894741,12.557000,19.260524,17.367967,20.696909,19.529036,14.146881,15.704053,11.436133,19.904397,21.664004,21.289135,17.695349,16.906900,16.502864,17.034669,17.955119
min,0.000000,16.000000,16.000000,46.000000,48.000000,731.000000,1.000000,1.000000,1.000000,1.000000,5.000000,2.000000,4.000000,7.000000,4.000000,4.000000,6.000000,3.000000,9.000000,5.000000,12.000000,12.000000,14.000000,21.000000,16.000000,2.000000,15.000000,12.000000,17.000000,3.000000,11.000000,3.000000,2.000000,10.000000,5.000000,3.000000,3.000000,2.000000,3.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,4551.500000,200315.500000,21.000000,62.000000,67.000000,1457.000000,1.000000,3.000000,2.000000,8.000000,38.000000,30.000000,44.000000,54.000000,30.000000,49.000000,34.000000,31.000000,43.000000,54.000000,57.000000,57.000000,55.000000,56.000000,56.000000,45.000000,58.000000,56.000000,58.000000,33.000000,44.000000,26.000000,38.000000,44.000000,39.000000,51.000000,30.000000,27.000000,24.000000,8.000000,8.000000,8.000000,8.000000,8.000000
50%,9103.000000,221759.000000,25.000000,66.000000,71.000000,1635.000000,1.000000,3.000000,2.000000,17.000000,54.000000,49.000000,56.000000,62.000000,44.000000,61.000000,48.000000,41.000000,56.000000,63.000000,67.000000,67.000000,66.000000,62.000000,66.000000,59.000000,66.000000,66.000000,67.000000,51.000000,59.000000,52.000000,55.000000,55.000000,49.000000,60.000000,53.000000,55.000000,52.000000,11.000000,11.000000,11.000000,11.000000,11.000000
75%,13654.500000,236529.500000,28.000000,71.000000,75.000000,1787.000000,1.000000,3.000000,3.000000,26.000000,64.000000,62.000000,64.000000,68.000000,57.000000,68.000000,62.000000,57.000000,64.000000,69.000000,75.000000,75.000000,74.000000,68.000000,74.000000,68.000000,73.000000,74.000000,74.000000,62.000000,69.000000,64.000000,64.000000,64.000000,60.000000,67.000000,64.000000,66.000000,64.000000,14.000000,14.000000,14.000000,14.000000,14.000000
max,18206.000000,246620.000000,45.000000,94.000000,95.000000,2346.000000,5.000000,5.000000,5.000000,99.000000,93.000000,95.000000,94.000000,93.000000,90.000000,97.000000,94.000000,94.000000,93.000000,96.000000,97.000000,96.000000,96.000000,96.000000,96.000000,95.000000,95.000000,96.000000,97.000000,94.000

## Neteja de dades
Ara que ja tenim una noció sobre les dades que tenim disponibles al `dataframe`, podem passar a "netejar-les" per quedar-nos només amb les dades que ens interessin i en un format adient pel model de predicció que hem de desenvolupar.

En primer lloc, podem eliminar les dues primeres columnes, que com hem vist, es corresponen a dos identificadors numèrics que no aporten cap informació.

In [7]:
df = df.iloc[:, 2:]

### Club
En primer lloc, ens encarregarem de la columna **Club**. Podem veure un llistat dels jugadors que tenen el valor NaN (Not a number) en aquesta columna. El motiu és que el club d'aquests jugadors no està a la base de dades de FIFA.

In [8]:
df.loc[df.Club != df.Club, ["Name", "Age", "Nationality", "Overall", "Value", "Club"]]

,Name,Age,Nationality,Overall,Value,Club
452,L. Paredes,24,Argentina,80,€0,NaN
538,A. Granqvist,33,Sweden,80,€0,NaN
568,A. Lunev,26,Russia,79,€0,NaN
677,I. Smolnikov,29,Russia,79,€0,NaN
874,A. Dzyuba,29,Russia,78,€0,NaN
...,...,...,...,...,...,...
17197,V. Kaith,21,India,55,€0,NaN
17215,K. Taimi,26,Finland,55,€0,NaN
17339,S. Passi,23,India,54,€0,NaN
17436,D. Lalhlimpuia,20,India,54,€0,NaN


Com només $241$ dels $18207$ jugadors que té la base de dades tenen el valor de Club a NaN, eliminarem aquests registres.

In [9]:
df = df.dropna(subset = ["Club"])

Ara, per poder passar d'una variable de tipus text com és el **Club** a una variable numèrica, emprarem la notació [One-hot](https://en.wikipedia.org/wiki/One-hot). Aquest tipus de notació consisteix a crear una categoria nova per a cada club. Si el jugador pertany a un club determinat, hi haurà un 1 en la columna d'aquell club.

A continuació es troben les instruccions per convertir les dades a la notació especificada:

In [10]:
clb = df.pop("Club")

df = pd.concat([df.reset_index(drop=True), pd.get_dummies(clb, prefix="clb").reset_index(drop=True)], 
               axis=1, sort=False)
df.loc[:5, ["Name", "clb_FC Barcelona", "clb_Juventus", "clb_Paris Saint-Germain", 
            "clb_Manchester United", "clb_Manchester City"]]

,Name,clb_FC Barcelona,clb_Juventus,clb_Paris Saint-Germain,clb_Manchester United,clb_Manchester City
0,L. Messi,1,0,0,0,0
1,Cristiano Ronaldo,0,1,0,0,0
2,Neymar Jr,0,0,1,0,0
3,De Gea,0,0,0,1,0
4,K. De Bruyne,0,0,0,0,1
5,E. Hazard,0,0,0,0,0


### Value

Hem de convertir els valors de la columna **Value** (valor monetari d'un jugador) del format text a un format numèric. Per a això, definirem una funció que converteixi un valor monetari representat com un text al valor en format `float`, i aplicarem aquesta funció a tots els valor de la columna **Value**.

In [11]:
def value_to_float(x):
    """
    From K and M to float.
    
    """
    x = x.replace("€", "")
    ret_val = 0.0
    
    if type(x) == float or type(x) == int:
        ret_val = x
    if "K" in x:
        if len(x) > 1:
            ret_val = float(x.replace("K", ""))
        ret_val = ret_val *1000
    if "M" in x:
        if len(x) > 1:
            ret_val = float(x.replace("M", ""))
        ret_val = ret_val * 1000000.0
    return ret_val

In [12]:
df["Value"] = df["Value"].apply(value_to_float)

### Wage

En el cas dels salaris dels jugadors, el valor monetari està en el mateix format que en el cas del valor de mercat. Per tant, podem arreglar les dades aplicant la mateixa funció `value_to_float`.

In [13]:
df["Wage"] = df["Wage"].apply(value_to_float)

### Preferred Foot
Per la columna **Preferred Foot**, que només té dues opcions disponibles ("Left" o "Right") emprarem la notació One-hot, com en el cas dels clubs.

In [14]:
pref_foot = df.pop("Preferred Foot")

df = pd.concat([df.reset_index(drop=True), pd.get_dummies(pref_foot, prefix="pref_foot").reset_index(drop=True)], axis=1, sort=False)
df.loc[[0, 1, 2, 5], ["Name", "pref_foot_Left", "pref_foot_Right"]]

,Name,pref_foot_Left,pref_foot_Right
0,L. Messi,1,0
1,Cristiano Ronaldo,0,1
2,Neymar Jr,0,1
5,E. Hazard,0,1


### Position
De manera similar, convertirem la columna **Position** en tantes categories com posicions al terreny de joc hi ha amb la notació One-hot.

In [15]:
pos = df.pop("Position")

df = pd.concat([df.reset_index(drop=True), pd.get_dummies(pos, prefix="pos").reset_index(drop=True)], axis=1, sort=False)
df.loc[[10, 11, 12, 13, 14], ["Name", "Age", "Nationality", "Overall", "Value",
                              "pos_CB", "pos_LCB", "pos_LCM", "pos_LDM", "pos_ST"]]

,Name,Age,Nationality,Overall,Value,pos_CB,pos_LCB,pos_LCM,pos_LDM,pos_ST
10,R. Lewandowski,29,Poland,90,77000000.0,0,0,0,0,1
11,T. Kroos,28,Germany,90,76500000.0,0,0,1,0,0
12,D. Godín,32,Uruguay,90,44000000.0,1,0,0,0,0
13,David Silva,32,Spain,90,60000000.0,0,0,1,0,0
14,N. Kanté,27,France,89,63000000.0,0,0,0,1,0


### Height
Pel cas de l'altura d'un jugador, que ve representada al `dataframe` com un string amb l'altura en el [sistema imperial](https://en.wikipedia.org/wiki/Imperial_units), la convertirem al sistema mètric (en centímetres) emprant la següent funció).

In [16]:
def imperial_to_metric(ht):
    # format: 7' 0.0"
    ht_ = ht.split("'")
    ft_ = float(ht_[0])
    in_ = float(ht_[1])
    return int(30.48 * ft_ + 2.54 * in_)

In [17]:
df = df.dropna(subset = ["Height"])
df["Height"] = df["Height"].apply(imperial_to_metric)

Una vegada aplicada la funció a tots els valors de la columna **Height**, podem veure el valor de les altures de cada jugador figura ara en centímetres.

In [18]:
df.loc[[0, 7, 19, 65, 76], ["Name", "Age", "Nationality", "Height"]]

,Name,Age,Nationality,Height
0,L. Messi,31,Argentina,170
7,L. Suárez,31,Uruguay,182
19,T. Courtois,26,Belgium,198
65,Douglas Costa,27,Brazil,172
76,Iniesta,34,Spain,170


### Weight
En el cas del pes, que està representat com un `string` amb el format "500 lbs", convertirem aquest valor a un nombre sencer amb la funció `weight_to_int`:

In [19]:
def weight_to_int(w):
    # format: 500 lbs
    w = w[:-3:]
    return int(w)

In [20]:
df["Weight"] = df["Weight"].apply(weight_to_int)
df.loc[[22, 23, 24, 25, 26], ["Name", "Age", "Nationality", "Weight"]]

,Name,Age,Nationality,Weight
22,M. Neuer,32,Germany,203
23,S. Agüero,30,Argentina,154
24,G. Chiellini,33,Italy,187
25,K. Mbappé,19,France,161
26,M. Salah,26,Egypt,157


### Release Clause
En el cas de la clàusula de rescissió, hi ha alguns jugadors que tenen aquest valor a NaN.

In [21]:
df.loc[df["Release Clause"] != df["Release Clause"]][["Name", "Age", "Height", "Weight", "Potential", "Value", "Wage", "Release Clause"]]

,Name,Age,Height,Weight,Potential,Value,Wage,Release Clause
28,J. Rodríguez,26,180,172,89,69500000.0,315000.0,NaN
38,G. Higuaín,30,185,196,88,57000000.0,245000.0,NaN
91,Paulinho,29,182,157,85,37000000.0,235000.0,NaN
166,Anderson Talisca,24,190,176,90,36500000.0,18000.0,NaN
176,M. Kovačić,24,175,172,89,35000000.0,165000.0,NaN
...,...,...,...,...,...,...,...,...
17738,Zhang Yuan,21,182,172,60,60000.0,2000.0,NaN
17785,Chen Ji,21,177,143,59,50000.0,2000.0,NaN
17790,Zhou Xin,20,170,143,61,40000.0,2000.0,NaN
17815,L. Valeau,19,177,143,65,60000.0,3000.0,NaN


Això és degut al fet que hi ha alguns països on no és obligatori que un jugador tengui clàusula de rescissió. Això vol dir que no seria correcte posar la clàusula a 0, ja que això indicaria que el jugador té un valor baix, i no té per què ser així. Pot haver-hi jugadors amb clàusula amb menor valor que jugadors sense clàusula i viceversa.

Per tant, el que farem serà eliminar els jugadors sense clàusula de rescissió, i a la resta, convertir la clàusula a un valor numèric.

In [22]:
df = df.dropna(subset = ["Release Clause"])
df["Release Clause"] = df["Release Clause"].apply(value_to_float)
df.loc[[13, 98, 123, 145], ["Name", "Age", "Nationality", "Release Clause"]]

,Name,Age,Nationality,Release Clause
13,David Silva,32,Spain,111000000.0
98,E. Džeko,32,Bosnia Herzegovina,51900000.0
123,Carvajal,26,Spain,66900000.0
145,Jonas,34,Brazil,33000000.0


## Predicció
Ara que ja tenim totes les columnes que volem emprar per a la predicció amb les dades netejades, eliminarem les columnes que no ens interessen per a la predicció.

Les columnes que sí que emprarem per a la predicció són les següents:
* Club (One-hot)
* Preferred Foot (One-hot)
* Position (One-hot)
* GKDiving
* GKHandling
* GKKicking
* GKPositioning
* GKReflexes
* Composture
* Marking
* StandingTackle
* Overall
* Potential
* Age
* Wage
* Weak Foot
* Skill Moves
* Height
* Weight
* Release Clause

In [23]:
val = df.pop("Value")
allowed_cols = ["Composture", "Marking", "StandingTackle", "Overall", "Potential", "Age", "Wage", "Weak Foot", "Skill Moves", "Height", "Weight", "Release Clause"]
to_drop = [c for c in df.columns if not c.startswith("clb") and not c.startswith("pref_foot") and not c.startswith("pos_") and not c.startswith("GK") and not c in allowed_cols]

df = df.drop(to_drop, axis=1)
df

,Age,Overall,Potential,Wage,Weak Foot,Skill Moves,Height,Weight,Marking,StandingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause,clb_ SSV Jahn Regensburg,clb_1. FC Heidenheim 1846,clb_1. FC Kaiserslautern,clb_1. FC Köln,clb_1. FC Magdeburg,clb_1. FC Nürnberg,clb_1. FC Union Berlin,clb_1. FSV Mainz 05,clb_AC Ajaccio,clb_AC Horsens,clb_AD Alcorcón,clb_ADO Den Haag,clb_AEK Athens,clb_AFC Wimbledon,clb_AIK,clb_AJ Auxerre,clb_AS Béziers,clb_AS Monaco,clb_AS Nancy Lorraine,clb_AS Saint-Étienne,clb_AZ Alkmaar,clb_Aalborg BK,clb_Aarhus GF,clb_Aberdeen,clb_Accrington Stanley,clb_Adelaide United,clb_Ajax,clb_Akhisar Belediyespor,clb_Al Ahli,clb_Al Batin,clb_Al Faisaly,clb_Al Fateh,clb_Al Fayha,clb_Al Hazem,clb_Al Hilal,clb_Al Ittihad,clb_Al Nassr,clb_Al Qadisiyah,clb_Al Raed,clb_Al Shabab,clb_Al Taawoun,clb_Al Wehda,clb_Alanyaspor,clb_Albacete BP,clb_Alianza Petrolera,clb_Amiens SC,clb_América FC (Minas Gerais),clb_América de Cali,clb_Angers SCO,clb_Antalyaspor,clb_Argentinos Juniors,clb_Arka Gdynia,clb_Arsenal,clb_Ascoli,clb_Aston Villa,clb_Atalanta,clb_Athletic Club de Bilbao,clb_Atiker Konyaspor,clb_Atlanta United,clb_Atlético Bucaramanga,clb_Atlético Huila,clb_Atlético Madrid,clb_Atlético Mineiro,clb_Atlético Nacional,clb_Atlético Paranaense,clb_Atlético Tucumán,clb_Audax Italiano,clb_BB Erzurumspor,clb_BK Häcken,clb_BSC Young Boys,clb_Bahia,clb_Barnsley,clb_Bayer 04 Leverkusen,clb_Beijing Renhe FC,clb_Beijing Sinobo Guoan FC,clb_Belgrano de Córdoba,clb_Benevento,clb_Beşiktaş JK,clb_Birmingham City,clb_Blackburn Rovers,clb_Blackpool,clb_Boavista FC,clb_Boca Juniors,clb_Bohemian FC,clb_Bologna,clb_Bolton Wanderers,clb_Borussia Dortmund,clb_Borussia Mönchengladbach,clb_Botafogo,clb_Bournemouth,clb_Boyacá Chicó FC,clb_Bradford City,clb_Bray Wanderers,clb_Brentford,clb_Brescia,clb_Brighton & Hove Albion,clb_Brisbane Roar,clb_Bristol City,clb_Bristol Rovers,clb_Brøndby IF,clb_Burnley,clb_Bursaspor,clb_Burton Albion,clb_Bury,clb_CA Osasuna,clb_CD Antofagasta,clb_CD Aves,clb_CD Everton de Viña del Mar,clb_CD Feirense,clb_CD Huachipato,clb_CD Leganés,clb_CD Lugo,clb_CD Nacional,clb_CD Numancia,clb_CD O'Higgins,clb_CD Palestino,clb_CD Tenerife,clb_CD Tondela,clb_CD Universidad de Concepción,clb_CF Rayo Majadahonda,clb_CF Reus Deportiu,clb_Cagliari,clb_Cambridge United,clb_Cardiff City,clb_Carlisle United,clb_Carpi,clb_Ceará Sporting Club,clb_Celtic,clb_Central Coast Mariners,clb_Cerezo Osaka,clb_Chamois Niortais Football Club,clb_Changchun Yatai FC,clb_Chapecoense,clb_Charlton Athletic,clb_Chelsea,clb_Cheltenham Town,clb_Chicago Fire,clb_Chievo Verona,clb_Chongqing Dangdai Lifan FC SWM Team,clb_Cittadella,clb_Clermont Foot 63,clb_Club América,clb_Club Atlas,clb_Club Atlético Aldosivi,clb_Club Atlético Banfield,clb_Club Atlético Colón,clb_Club Atlético Huracán,clb_Club Atlético Lanús,clb_Club Atlético Talleres,clb_Club Atlético Tigre,clb_Club Brugge KV,clb_Club Deportes Temuco,clb_Club León,clb_Club Necaxa,clb_Club Tijuana,clb_Clube Sport Marítimo,clb_Colchester United,clb_Colo-Colo,clb_Colorado Rapids,clb_Columbus Crew SC,clb_Cork City,clb_Cosenza,clb_Coventry City,clb_Cracovia,clb_Crawley Town,clb_Crewe Alexandra,clb_Crotone,clb_Cruz Azul,clb_Cruzeiro,clb_Crystal Palace,clb_Curicó Unido,clb_Cádiz CF,clb_Córdoba CF,clb_DC United,clb_DSC Arminia Bielefeld,clb_Daegu FC,clb_Dalian YiFang FC,clb_Dalkurd FF,clb_De Graafschap,clb_Defensa y Justicia,clb_Deportes Iquique,clb_Deportes Tolima,clb_Deportivo Alavés,clb_Deportivo Cali,clb_Deportivo Pasto,clb_Deportivo Toluca,clb_Deportivo de La Coruña,clb_Derby County,clb_Derry City,clb_Dijon FCO,clb_Dinamo Zagreb,clb_Djurgårdens IF,clb_Doncaster Rovers,clb_Dundalk,clb_Dundee FC,clb_Dynamo Kyiv,clb_ESTAC Troyes,clb_Eintracht Braunschweig,clb_Eintracht Frankfurt,clb_Elche CF,clb_Empoli,clb_En Avant de Guingamp,clb_Envigado FC,clb_Esbjerg fB,clb_Estudiantes de La Plata,clb_Ettifaq FC,clb_Everton,clb_Excelsior,clb_Exeter City,clb_Extremadura UD,clb_FC Admira Wacker Mödling,clb_FC

Una vegada tenim la informació de cada jugador i el valor que volem predir, separarem aquestes dades.

In [24]:
X_train, X_test, y_train, y_test = train_test_split(df, val, test_size=0.33, random_state=42)

In [25]:
len(X_train)

11150

Entrenam un model de regressió lineal per a les dades que hem netejat:

In [26]:
reg = linear_model.LinearRegression().fit(X_train, y_train)

I finalment, per poder avaluar la correctesa del model, calcularem la mètrica $R^2$ de la regressió, emprant la implementació de <a href="https://scikit-learn.org/stable/modules/generated/sklearn.metrics.r2_score.html"> sickit-learn</a>.

In [27]:
preds = reg.predict(X_test)

In [28]:
preds[0]

4640924.939951897

In [29]:
y_test[0]

110500000.0

In [30]:
r2_score(preds, y_test)

0.993388422603713

Podem comprovar com la mètrica $R^2$ dóna un valor d'aproximadament $0.993$, el qual s'acosta molt al valor ideal, que és $1$.